In [ ]:
import numpy as np
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())

In [ ]:
def eksi_scrap(url):
    url = url
    driver.get(url)
    
    entry_item_list = driver.find_elements_by_xpath('//*[@id="entry-item-list"]')[0]
    
    entries = []
    authors = []
    dates = []
    
    for i in range(len(entry_item_list.find_elements_by_class_name("content"))):
        entries.append(entry_item_list.find_elements_by_class_name("content")[i].text)
        authors.append(entry_item_list.find_elements_by_class_name("entry-author")[i].text)
        dates.append(entry_item_list.find_elements_by_class_name("entry-date")[i].text)
        
    df = pd.DataFrame(columns = ["Entry", "Author", "Date"])
    
    entries = pd.Series(entries, name='Entry')
    authors = pd.Series(authors, name='Author')
    dates = pd.Series(dates, name='Date')
    
    df = df.append(pd.concat([entries, authors, dates], axis=1))
    return df

In [ ]:
def get_entries(url):
    print(f"Getting Entries in: {url}")
    driver.get(url)
    
    # Check if the topic was changed to a new one
    try:
        if driver.find_element_by_xpath('//*[@id="title"]/sup/a'):
            old_topic = driver.find_element_by_xpath('//*[@id="title"]/sup/a').get_attribute("title")[1:-15]
    except:
        old_topic = driver.find_element_by_xpath('//*[@id="title"]/a/span').text
    
    if driver.find_element_by_xpath('//*[@id="entry-author"]/a').text == 'başlık taşıyan':
        url = driver.find_elements_by_xpath('//*[@id="entry-item-list"]')[0].find_element_by_class_name("content").find_element_by_class_name("b").get_attribute("href")
        driver.get(url)
        
    url = driver.current_url
    driver.get(url)
        
    if "true&rf" in url:
        url = driver.find_element_by_xpath('//*[@id="title"]/a').get_attribute("href")
        driver.get(url)
    
    new_topic = driver.find_element_by_xpath('//*[@id="title"]/a/span').text
    
    # Check if the topic has more than one page
    try:
        total_page = driver.find_element_by_xpath('//*[@id="topic"]/div[1]/div[2]/a[1]').text
    except:
        total_page = 1
    
    url = url + "?p="
    
    df = pd.DataFrame(columns = ["Entry", "Author", "Date"])
    
    i = 1
    while True:
        try:
            df = df.append(eksi_scrap(url + str(i)))
            print(f"Page: {i}|{total_page}")
            i += 1
        except:
            break
            
    df = df.reset_index(drop=True)
    old_topics = pd.Series([old_topic]*df.shape[0], name="Old_Topic")
    new_topics = pd.Series([new_topic]*df.shape[0], name="New_Topic")
    
    df = pd.concat([df, old_topics, new_topics], axis=1)
    return df

# Kadıköy Mahalle Entries


In [ ]:
kadıkoy = pd.DataFrame(columns = ["Entry", "Author", "Date"])

In [ ]:
urls = [""]

In [ ]:
for i, url in enumerate(urls):
    try:
        kadıkoy = kadıkoy.append(get_entries(url))
    except:
        kadıkoy = kadıkoy.append(get_entries(url))

In [ ]:
kadıkoy.head()

In [ ]:
kadıkoy = kadıkoy.reset_index(drop=True)